In [ ]:
import pandas as pd
import os
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore")

In [2]:
news = pd.read_csv("NewsData/BNB_News_Data.csv").drop("Unnamed: 0", axis = 1)
stocks = pd.read_csv("StocksData/BNB_Stocks_Scrapped.csv").drop("Unnamed: 0", axis = 1)

In [3]:
news

,Crypto,Title,Date
0,binance cypto,Singapore Government: This is How We Intend to...,2014-01-09
1,binance cypto,From Arvind Kejriwal's muffler to Modi's kurta...,2014-01-28
2,binance cypto,Apple Removes Blockchain Bitcoin Wallet Apps f...,2014-02-06
3,binance cypto,Dorian S. Nakamoto Denies He's Bitcoin Founder,2014-03-07
4,binance cypto,Federal Judge Freezes US Assets Of Mt Gox CEO ...,2014-03-12
...,...,...,...
7277,binance cypto,Worksport Embraces Bitcoin and XRP to Strength...,2024-12-05
7278,binance cypto,15 Cryptocurrency Forecasts For 2025,2024-12-05
7279,binance cypto,Virgin Voyages Introduces 'Industry-First' Bit...,2024-12-03
7280,binance cypto,Meme Coins Go Mainstream: Coinbase Lists GIGA ...,2024-12-04


In [4]:
stocks

,Date,Open,High,Low,Close,Adj Close,Volume
0,"Nov 2, 2024",572.75,575.68,572.75,575.68,575.68,"1,629,498,240"
1,"Oct 31, 2024",595.73,596.07,571.43,576.03,576.03,"1,810,695,051"
2,"Oct 30, 2024",605.67,606.07,594.45,595.73,595.73,"1,662,620,787"
3,"Oct 29, 2024",600.60,611.78,599.95,605.67,605.67,"1,886,358,424"
4,"Oct 28, 2024",588.72,603.72,585.12,600.60,600.60,"1,671,332,663"
...,...,...,...,...,...,...,...
2545,"Nov 13, 2017",1.53,1.74,1.52,1.69,1.69,"12,238,800"
2546,"Nov 12, 2017",1.67,1.67,1.46,1.52,1.52,"15,298,700"
2547,"Nov 11, 2017",1.79,1.92,1.61,1.67,1.67,"8,178,150"
2548,"Nov 10, 2017",2.01,2.07,1.64,1.80,1.80,"11,155,000"


In [5]:
stocks['Date'] = pd.to_datetime(stocks['Date']).dt.strftime('%Y-%m-%d')

In [6]:
stocks

,Date,Open,High,Low,Close,Adj Close,Volume
0,2024-11-02,572.75,575.68,572.75,575.68,575.68,"1,629,498,240"
1,2024-10-31,595.73,596.07,571.43,576.03,576.03,"1,810,695,051"
2,2024-10-30,605.67,606.07,594.45,595.73,595.73,"1,662,620,787"
3,2024-10-29,600.60,611.78,599.95,605.67,605.67,"1,886,358,424"
4,2024-10-28,588.72,603.72,585.12,600.60,600.60,"1,671,332,663"
...,...,...,...,...,...,...,...
2545,2017-11-13,1.53,1.74,1.52,1.69,1.69,"12,238,800"
2546,2017-11-12,1.67,1.67,1.46,1.52,1.52,"15,298,700"
2547,2017-11-11,1.79,1.92,1.61,1.67,1.67,"8,178,150"
2548,2017-11-10,2.01,2.07,1.64,1.80,1.80,"11,155,000"


In [7]:
df = pd.merge(news, stocks, on = 'Date', how = 'right').drop('Crypto', axis = 1)
df

,Title,Date,Open,High,Low,Close,Adj Close,Volume
0,The Rise and Fall of CZ: Changpeng Zhao’s Jour...,2024-11-02,572.75,575.68,572.75,575.68,575.68,"1,629,498,240"
1,Coinbase slumps 10% after missing key Q3 earni...,2024-10-31,595.73,596.07,571.43,576.03,576.03,"1,810,695,051"
2,Bitcoin's October Performance Shows Significan...,2024-10-30,605.67,606.07,594.45,595.73,595.73,"1,662,620,787"
3,Bitcoin teases all-time high days from preside...,2024-10-30,605.67,606.07,594.45,595.73,595.73,"1,662,620,787"
4,Bitcoin Profit-Taking Continues as BTC Price N...,2024-10-30,605.67,606.07,594.45,595.73,595.73,"1,662,620,787"
...,...,...,...,...,...,...,...,...
6991,Bitcoin by Numbers: 21 Statistics That Reveal ...,2017-11-11,1.79,1.92,1.61,1.67,1.67,"8,178,150"
6992,The Satoshi Revolution - Chapter 2: Was Satosh...,2017-11-11,1.79,1.92,1.61,1.67,1.67,"8,178,150"
6993,NaN,2017-11-10,2.01,2.07,1.64,1.80,1.80,"11,155,000"
6994,Samantha Chang,2017-11-09,2.05,2.17,1.89,1.99,1.99,"19,192,200"


In [8]:
df['Date'].nunique()

2550

In [9]:
df['Title'].isnull().sum()

407

In [10]:
df[df['Title'].isnull()]

,Title,Date,Open,High,Low,Close,Adj Close,Volume
66,NaN,2024-10-13,576.45,578.06,565.13,571.82,571.82,"1,472,484,694"
67,NaN,2024-10-12,572.86,578.71,572.86,576.45,576.45,"1,482,994,534"
140,NaN,2024-09-21,568.58,590.03,566.81,589.04,589.04,"1,694,887,390"
202,NaN,2024-09-02,512.60,529.02,502.17,526.44,526.44,"1,653,356,892"
203,NaN,2024-09-01,532.90,532.98,508.72,512.60,512.60,"1,498,350,087"
...,...,...,...,...,...,...,...,...
6978,NaN,2017-11-19,1.67,1.70,1.57,1.61,1.61,"38,081,700"
6979,NaN,2017-11-18,1.51,1.68,1.50,1.67,1.67,"32,167,300"
6985,NaN,2017-11-15,1.59,1.62,1.50,1.53,1.53,"7,615,500"
6986,NaN,2017-11-14,1.69,1.74,1.57,1.59,1.59,"7,829,600"


In [ ]:
os.environ["GROQ_API_KEY"] = "Enter API Key Here"
os.environ["OPENAI_API_KEY"] = "Enter API Key Here"

In [12]:
llm = ChatGroq(model = "llama-3.3-70b-versatile")

# Define the memory object
memory = ConversationBufferMemory()

# Define the system prompt
system_message = (
    f"""
    - You are a machine learning bot that has background in financial news, stock news, political science, cryptocurrencies, and international relations that can affect stock prices
    - Analyze the following news title and identify the persuasion techniques used from the list provided below. 
    - Give a confidence score (on a scale of 0 to 1) how likely it is to be using one of the persuasion techniques mentioned below.
    - The confidence metric should indicate how confident you are that the identified technique is present in the title. 
    - Consider the specific clues related to each technique to aid in your analysis.

    **Possible Persuasion Techniques:**
    1. Appeal_to_Authority: Cites authority to support a conclusion.
    2. Appeal_to_Popularity: Cites popularity or majority support to back a claim.
    3. Appeal_to_Values: Invokes widely shared values.
    4. Appeal_to_Fear-Prejudice: Uses fear or prejudice to reject or promote an idea.
    5. Flag_Waving: Refers to patriotism or group allegiance.
    6. Causal_Oversimplification: Oversimplifies the causes of an issue.
    7. False_Dilemma-No_Choice: Implies only two options when there may be more.
    8. Consequential_Oversimplification: Oversimplifies the consequences of accepting a proposition.
    9. Straw_Man: Misrepresents someone's position to make it easier to attack.
    10. Red_Herring: Diverts attention from the main topic.
    11. Whataboutism: Distracts from the topic by charging an opponent with hypocrisy.
    12. Slogans: Uses a brief, catchy phrase to encapsulate a message.
    13. Appeal_to_Time: Suggests that the time is ripe for a certain action.
    14. Conversation_Killer: Discourages critical thought or discussion.
    15. Loaded_Language: Uses emotionally charged words or phrases.
    16. Repetition: Repeatedly reinforces the same idea.
    17. Exaggeration-Minimisation: Downplays or exaggerates a subject.
    18. Obfuscation-Vagueness-Confusion: Deliberately unclear, leaving room for varied interpretations.
    19. Name_Calling-Labeling: Employs demeaning labels.
    20. Doubt: Undermines credibility by questioning character or attributes.
    21. Guilt_by_Association: Discredits by associating with a negatively viewed group.
    22. Appeal_to_Hypocrisy: Accuses the target of hypocrisy.
    23. Questioning_the_Reputation: Undermines the reputation of the target.
    24. None: The text appears unbiased and doesn't evidently employ persuasion techniques.

    Input and Output constraints:
    Input: text: string
    Output: confidence_score: float
    
    **Example Input:**  
    NaN

    **Example Output:**
    0

    **Example Input:**
    Foreign Secretary Jeremy Hunt says the UK should not 'underestimate its strength' after Brexit
    
    **Example Output:**
    0.8
    
    **Example Input with No Persuasion Technique:**
    The Parliamentary vote on British Prime Minister Theresa May’s Brexit plan is scheduled to be held next month. The outcome of the vote is uncertain as opinions within Parliament are divided.

    **Example Output with No Persuasion Technique:**
    0
    
    Please Note: I only need the confidence metrics relating to the title 
    I dont need any kind of explanation in the output. Also note you should never give any answer apart from the format provided above, nothing else!
    """
)

# Define prompt templates
system_prompt_template = SystemMessagePromptTemplate.from_template(system_message)
human_prompt_template = HumanMessagePromptTemplate.from_template("{input}")

chat_prompt = ChatPromptTemplate.from_messages([system_prompt_template, human_prompt_template])


# Create the chain
chain = LLMChain(
    llm=llm,
    prompt=chat_prompt,
    memory=memory
)

def get_persuasion(text) -> list:
    if text is None:
        return 0
        
    response = chain.run(input=text)

    return response

In [13]:
t1 = df['Title'][21]
t1

'HOW TO EARN $10 TO $20 DAILY ON BINANCE AS A BEGINNER😎👇🏻'

In [14]:
print(get_persuasion(t1))

0.9


In [15]:
import numpy as np
# Corrected implementation
lens = [len(t) for t in df['Title'] if pd.notna(t)]
np.mean(lens)

66.90514493853392

In [16]:
tdf = df.sample(n=10)
from tqdm import tqdm

# Enable tqdm for pandas apply
tqdm.pandas()

# Apply the function with progress tracking
tdf['persuasion_score'] = tdf['Title'].progress_apply(get_persuasion)

# tdf['persuasion_score'] = tdf['Title'].apply(get_persuasion)

100%|██████████| 10/10 [00:24<00:00,  2.42s/it]


In [17]:
tdf

,Title,Date,Open,High,Low,Close,Adj Close,Volume,persuasion_score
2673,Nigeria To Establish Special Economic Zone For...,2022-09-06,276.75,285.75,262.60,262.67,262.67,"1,173,656,468",0.2
5798,Binance’s total monthly volume of 42.1 billion...,2019-08-04,27.88,28.07,27.36,27.56,27.56,"136,844,756",0.7
3456,Russia proposes ban on use and mining of crypt...,2022-01-20,462.86,478.35,439.98,440.01,440.01,"1,881,741,995",0.2
3388,Binance CEO Changpeng 'CZ' Zhao says regulator...,2022-02-07,419.67,438.64,414.25,433.23,433.23,"1,755,674,427",0.2
2735,Growing Crypto Through Sign Language: Life as ...,2022-08-16,319.71,322.16,312.81,316.42,316.42,"963,072,351",0.2
1124,Ronin Crypto Wallet Integrates Binance Pay for...,2023-12-03,229.29,231.64,226.50,228.10,228.10,"719,496,781",0.0
4781,US Bank Quontic Launches Bitcoin Rewards Check...,2020-12-16,29.55,30.60,29.06,30.49,30.49,"347,039,607",0.2
6993,NaN,2017-11-10,2.01,2.07,1.64,1.80,1.80,"11,155,000",0
2351,Binance Generates 90% of Revenue From Transact...,2022-12-07,290.38,290.49,282.02,284.28,284.28,"759,320,494",0.2
840,The end of the crypto winter looms: These are ...,2024-02-27,401.64,403.74,390.92,395.00,395.00,"1,750,697,179",0.7


In [18]:
import openai

In [19]:
openai.api_key = os.getenv("OPENAI_API_KEY")

def get_persuasion_gpt(text):
    if text is np.nan:
        return 0.0
    try:
        # Replace "gpt-4o-mini" with a valid model name (e.g., "gpt-4" or "gpt-3.5-turbo")
        completion = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": text}
            ]
        )
        # Extract the response content
        return completion['choices'][0]['message']['content']
    except Exception as e:
        # Log and return the error for debugging
        print(f"Error processing text: {text}\n{e}")
        return 0.0


In [21]:
tdf = df.sample(n=10)
from tqdm import tqdm

# Enable tqdm for pandas apply
tqdm.pandas()

# Apply the function with progress tracking
tdf['persuasion_score'] = tdf['Title'].progress_apply(get_persuasion_gpt)

# tdf['persuasion_score'] = tdf['Title'].apply(get_persuasion)

100%|██████████| 10/10 [00:03<00:00,  2.83it/s]


In [22]:
tdf

,Title,Date,Open,High,Low,Close,Adj Close,Volume,persuasion_score
6474,‘TELL YOUR DAD TO GIVE US BITCOIN:’ How a Hack...,2018-07-30,13.95,13.95,13.07,13.22,13.22,"35,991,600",0.7
6160,NaN,2019-01-26,6.75,7.04,6.69,7.01,7.01,"48,498,979",0.0
811,New ruling in SEC’s Coinbase insider trading l...,2024-03-03,410.98,416.63,405.09,414.38,414.38,"1,810,575,392",0.7
1395,20 Countries that Use Crypto and Bitcoin the Most,2023-09-18,216.47,221.01,215.32,216.00,216.00,"572,231,452",0.4
4410,Robinhood Adds 9.5 Million New Crypto Users As...,2021-04-12,526.70,637.44,516.28,598.72,598.72,"14,436,315,725",0.7
6888,Top 13 Bitcoin faucets,2018-01-02,8.46,9.12,8.22,8.84,8.84,"104,789,000",0
3459,Bank of Russia Calls for Full Ban on Crypto,2022-01-20,462.86,478.35,439.98,440.01,440.01,"1,881,741,995",0.7
1767,Know Your Scam: How to Protect Your Crypto Fro...,2023-05-30,311.81,314.28,311.31,311.68,311.68,"392,038,878",0.7
3369,Crypto exchange Binance to invest $200 mln in ...,2022-02-10,422.51,434.17,411.25,415.12,415.12,"2,037,888,603",0
1177,"What is Binance, why is it in so much trouble,...",2023-11-21,253.64,269.92,224.98,226.49,226.49,"2,480,554,399",0.7


In [23]:
df['persuasion_score'] = df['Title'].progress_apply(get_persuasion_gpt)

100%|██████████| 6996/6996 [50:50<00:00,  2.29it/s]  


In [24]:
df

,Title,Date,Open,High,Low,Close,Adj Close,Volume,persuasion_score
0,The Rise and Fall of CZ: Changpeng Zhao’s Jour...,2024-11-02,572.75,575.68,572.75,575.68,575.68,"1,629,498,240",0.7
1,Coinbase slumps 10% after missing key Q3 earni...,2024-10-31,595.73,596.07,571.43,576.03,576.03,"1,810,695,051",0.3
2,Bitcoin's October Performance Shows Significan...,2024-10-30,605.67,606.07,594.45,595.73,595.73,"1,662,620,787",0.2
3,Bitcoin teases all-time high days from preside...,2024-10-30,605.67,606.07,594.45,595.73,595.73,"1,662,620,787",0.7
4,Bitcoin Profit-Taking Continues as BTC Price N...,2024-10-30,605.67,606.07,594.45,595.73,595.73,"1,662,620,787",0.7
...,...,...,...,...,...,...,...,...,...
6991,Bitcoin by Numbers: 21 Statistics That Reveal ...,2017-11-11,1.79,1.92,1.61,1.67,1.67,"8,178,150",0.7
6992,The Satoshi Revolution - Chapter 2: Was Satosh...,2017-11-11,1.79,1.92,1.61,1.67,1.67,"8,178,150",0.3
6993,NaN,2017-11-10,2.01,2.07,1.64,1.80,1.80,"11,155,000",0.0
6994,Samantha Chang,2017-11-09,2.05,2.17,1.89,1.99,1.99,"19,192,200",0


In [25]:
df.to_csv("BNB_persuasion_gpt.csv")